CMPE 266-01

Spring 2023

Team: Phillip Nguyen, Xuewei Zheng, Roger Kuo

Movie Recommendation System (using multi-dimensional indexing)

# Initialization

In [1]:
# If using Google Colab
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd

MovieLens 1M Dataset found here: https://grouplens.org/datasets/movielens/1m/

In [50]:
# Change directory
directory = "gdrive/My Drive/Colab Notebooks/CMPE 266/"

movies = pd.read_csv(directory+"movies.dat", sep='::', names=['mID','title','genres'], engine='python', encoding="ISO-8859-1")
ratings = pd.read_csv(directory+"ratings.dat", sep='::', names=['uID','mID','rating','timestamp'], engine='python')
users = pd.read_csv(directory+"users.dat", sep='::', names=['uID','gender','age','occupation','zip'], engine='python')

# Data Preparation

## View datasets

Information on the tables and their columns: https://files.grouplens.org/datasets/movielens/ml-1m-README.txt

In [13]:
print(movies.shape)
movies.head()

(3883, 3)


,mID,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
print(ratings.shape)
ratings.head()

(1000209, 4)


,uID,mID,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [15]:
print(users.shape)
users.head()

(6040, 5)


,uID,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


## Preprocessing

In [32]:
from sklearn.preprocessing import LabelEncoder

### Movies table

Omit the title column. We will use it later.

Each movie can have multiple genres, sorted in alphabetical order. 
To simplify our process, we just use the first assigned genre.
Otherwise we will have to incorporate more feature vectors.
While this would work fine with machine learning, it is ineffiient for our purposes with multi-dimensional indexing.

In [45]:
# Omit title column
m_df = movies.copy(deep=True)
m_df = m_df[['mID','genres']]
# Simplify genres column
m_df['genres'] = m_df['genres'].map(lambda genres: genres.split('|')[0])

# Encode the genres as integers
mle = LabelEncoder()
m_df['genres'] = mle.fit_transform(m_df['genres'])

m_df.head()

{'Action': 0, 'Adventure': 1, 'Animation': 2, "Children's": 3, 'Comedy': 4, 'Crime': 5, 'Documentary': 6, 'Drama': 7, 'Fantasy': 8, 'Film-Noir': 9, 'Horror': 10, 'Musical': 11, 'Mystery': 12, 'Romance': 13, 'Sci-Fi': 14, 'Thriller': 15, 'War': 16, 'Western': 17}


,mID,genres
0,1,2
1,2,1
2,3,4
3,4,4
4,5,4


### Ratings table

Omit the timestamp column. This feature is uselesss.

Omit the rating column. We may use it later.

In [46]:
r_df = ratings.copy(deep=True)
r_df = r_df[['uID','mID']]
r_df.head()

,uID,mID
0,1,1193
1,1,661
2,1,914
3,1,3408
4,1,2355


### Users table

Because many ratings come from the same user, we may not want to use every user feature.
Otherwise, we may get recommendations from the same user.

In [79]:
# Omit occupation and zip columns
u_df = users.copy(deep=True)

# Omit occupation and zip columns
u_df = u_df[['uID','gender','age']]

# Omit zip column
# u_df = u_df[['uID','gender','age','occupation']]

# Keep all columns
# u_df = u_df[['uID','gender','age','occupation','zip']]

# Encode the genders as integers
ule = LabelEncoder()
u_df['gender'] = ule.fit_transform(u_df['gender'])

u_df.head()

{'F': 0, 'M': 1}


,uID,gender,age
0,1,0,1
1,2,1,56
2,3,1,25
3,4,1,45
4,5,1,25


### Combined table

Combine all three tables into one

In [80]:
result = pd.merge(r_df, m_df, left_on='mID', right_on='mID')
result = pd.merge(result, u_df, left_on='uID', right_on='uID')

# Omit uID and mID
result = result[['genres','gender','age']]
feature_count = result.shape[1]
print(result.shape)
result.head()

(1000209, 3)


,genres,gender,age
0,7,0,1
1,2,0,1
2,11,0,1
3,7,0,1
4,2,0,1


# Multi-dimensional Indexing

## KD Tree

Insert our combined table into a KD-Tree.

In [66]:
from sklearn.metrics.pairwise import distance_metrics
from sklearn.neighbors import KDTree

In [69]:
KDTree.valid_metrics

['euclidean',
 'l2',
 'minkowski',
 'p',
 'manhattan',
 'cityblock',
 'l1',
 'chebyshev',
 'infinity']

In [95]:
# Default similarity metric is minkowski.

# Create two trees for comparision
kdt2 = KDTree(result, leaf_size=2, metric='euclidean')

kdt4 = KDTree(result, leaf_size=4, metric='euclidean')

## LSH

In [ ]:
!pip install lshashpy3

In [84]:
from lshashpy3 import LSHash

In [92]:
# Create two lsh for comparison; w/ n-features
feature_count = result.shape[1]

#2-bit hash 
lsh2 = LSHash(2, feature_count)

# 4-bit hash
lsh4 = LSHash(4, feature_count)

In [93]:
for row in result.itertuples():
  # print(list(row)[1:])
  lsh2.index(list(row)[1:])
  lsh4.index(list(row)[1:])

# NN Querying

In [133]:
import time

In [153]:
# Get the corresponding movies from a list of indices
def get_recommendations(ind):
  print("\nRecommended movies:")
  for i in ind[0]:
    mID = ratings.iloc[i]['mID']
    movie = movies.loc[movies['mID'] == mID].iloc[0]['title']
    print(movie)

## Define our user(s)

In [97]:
# Show query input parameters
result.head(1)

,genres,gender,age
0,7,0,1


In [98]:
#Show genre mapping
print(dict(zip(mle.classes_, mle.transform(mle.classes_))))

{'Action': 0, 'Adventure': 1, 'Animation': 2, "Children's": 3, 'Comedy': 4, 'Crime': 5, 'Documentary': 6, 'Drama': 7, 'Fantasy': 8, 'Film-Noir': 9, 'Horror': 10, 'Musical': 11, 'Mystery': 12, 'Romance': 13, 'Sci-Fi': 14, 'Thriller': 15, 'War': 16, 'Western': 17}


In [99]:
# Show gender mapping
print(dict(zip(ule.classes_, ule.transform(ule.classes_))))

{'F': 0, 'M': 1}


Age and occupation mappings can be found here: https://files.grouplens.org/datasets/movielens/ml-1m-README.txt

under the USERS FILE DESCRIPTION section

In [103]:
# Define our user(s)
user1 = [13, 0, 18] # Female, age 18-24, looking for Romance films
user2 = [6, 1, 45] # Male, age 45-49, looking for Documentaries
user3 = [1, 1, 25] # Male, age 25-43, looking for Adventure films

# Define how many movies we want
K = 5

## KD Tree

In [209]:
# Cound time of KDTree query, and return indices of similar user/preference
def run_KD(tree, user, k):
  start = time.time()
  dist, ind = tree.query([user], k=k)
  end = time.time()
  print("Time: ", end - start)
  return ind, end - start

In [211]:
# Run query on user1, using KD-Tree with leaf-size=2
print("Run query on user1, using KD-Tree with leaf-size=2")
ind, k12 = run_KD(kdt2, user1, K)
get_recommendations(ind)

# Run query on user1, using KD-Tree with leaf-size=4
print("\nRun query on user1, using KD-Tree with leaf-size=4")
ind, k14 = run_KD(kdt4, user1, K)
get_recommendations(ind)

Run query on user1, using KD-Tree with leaf-size=2
Time:  0.0005922317504882812

Recommended movies:
Dark Crystal, The (1982)
All About My Mother (Todo Sobre Mi Madre) (1999)
Time Bandits (1981)
Game, The (1997)
Donnie Brasco (1997)

Run query on user1, using KD-Tree with leaf-size=4
Time:  0.00043320655822753906

Recommended movies:
Dark Crystal, The (1982)
Time Bandits (1981)
Braveheart (1995)
Game, The (1997)
Donnie Brasco (1997)


In [213]:
# Run query on user2, using KD-Tree with leaf-size=2
print("Run query on user2, using KD-Tree with leaf-size=2")
ind, k22 = run_KD(kdt2, user2, K)
get_recommendations(ind)

# Run query on user2, using KD-Tree with leaf-size=4
print("\nRun query on user2, using KD-Tree with leaf-size=4")
ind, k24 = run_KD(kdt4, user2, K)
get_recommendations(ind)

Run query on user2, using KD-Tree with leaf-size=2
Time:  0.0006022453308105469

Recommended movies:
Mystery Science Theater 3000: The Movie (1996)
Little Big Man (1970)
Haunting, The (1999)
Creepshow (1982)
Lost in Space (1998)

Run query on user2, using KD-Tree with leaf-size=4
Time:  0.00045180320739746094

Recommended movies:
Creepshow (1982)
Little Big Man (1970)
Lost in Space (1998)
Haunting, The (1999)
Mystery Science Theater 3000: The Movie (1996)


In [214]:
# Run query on user3, using KD-Tree with leaf-size=2
print("Run query on user3, using KD-Tree with leaf-size=2")
ind, k32 = run_KD(kdt2, user3, K)
get_recommendations(ind)

# Run query on user3, using KD-Tree with leaf-size=4
print("\nRun query on user3, using KD-Tree with leaf-size=4")
ind, k34 = run_KD(kdt4, user3, K)
get_recommendations(ind)

Run query on user3, using KD-Tree with leaf-size=2
Time:  0.0028259754180908203

Recommended movies:
Natural Born Killers (1994)
Tender Mercies (1983)
Mission: Impossible (1996)
Star Wars: Episode IV - A New Hope (1977)
Abyss, The (1989)

Run query on user3, using KD-Tree with leaf-size=4
Time:  0.0015637874603271484

Recommended movies:
Star Wars: Episode IV - A New Hope (1977)
Fast, Cheap & Out of Control (1997)
Abyss, The (1989)
Mission: Impossible (1996)
Natural Born Killers (1994)


## LSH

In [223]:
# Cound time of LSH query
def run_lsh(lsh, user, k):
  start = time.time()
  nn = lsh2.query(user, num_results=K, distance_func="euclidean")
  end = time.time()
  print("Time: ", end - start)
  return nn, end - start

In [225]:
def get_indices(nn):
  ind = []
  for ((vec,extra_data),distance) in nn:
    print(vec)
    # row = result.loc[(result['genres'] == vec[0]) & (result['gender'] == vec[1]) & (result['age'] == vec[2])]
    # print(row)
  #   i = result.index(row)
  #   ind.append(i)
  # return ind

In [228]:
# Run query on user1, using LSH with 2-bit hash
print("Run query on user1, using LSH with 2-bit hash")
nn, l12 = run_lsh(lsh2, user1, K)
# ind = get_indices(nn)
# print(ind)
# get_recommendations(ind)

# Run query on user1, using LSH with 2-bit hash
print("\nRun query on user1, using LSH with 4-bit hash")
nn, l14 = run_lsh(lsh4, user1, K)

Run query on user1, using LSH with 2-bit hash
Time:  0.1289818286895752

Run query on user1, using LSH with 4-bit hash
Time:  0.11892843246459961


In [229]:
# Run query on user2, using LSH with 2-bit hash
print("Run query on user2, using LSH with 2-bit hash")
nn, l22 = run_lsh(lsh2, user2, K)

# Run query on user2, using LSH with 2-bit hash
print("\nRun query on user2, using LSH with 4-bit hash")
nn, l24 = run_lsh(lsh4, user2, K)

Run query on user2, using LSH with 2-bit hash
Time:  0.15179705619812012

Run query on user2, using LSH with 4-bit hash
Time:  0.11983203887939453


In [227]:
# Run query on user3, using LSH with 2-bit hash
print("Run query on user3, using LSH with 2-bit hash")
nn, l32 = run_lsh(lsh2, user3, K)

# Run query on user3, using LSH with 2-bit hash
print("\nRun query on user3, using LSH with 4-bit hash")
nn, l34 = run_lsh(lsh4, user3, K)

Run query on user3, using LSH with 2-bit hash
Time:  0.13401293754577637

Run query on user3, using LSH with 4-bit hash
Time:  0.11632561683654785


# Results / Analysis

Due to the limitations of our indexes and the number of total parameters we used, our recommendation system is neither optimal nor very accurate. 

However, our main goal is to measure the performances of the two different multidimensional indexes.

In [220]:
print("Using KD-Tree leaf-size 2, our NN query averaged:")
print((k12 + k22 + k32)/3,"seconds of runtime.")

Using KD-Tree leaf-size 2, our NN query averaged:
0.0013401508331298828 seconds of runtime.


In [221]:
print("Using KD-Tree leaf-size 4, our NN query averaged:")
print((k14 + k24 + k34)/3,"seconds of runtime.")

Using KD-Tree leaf-size 4, our NN query averaged:
0.0008162657419840494 seconds of runtime.


In [230]:
print("Using LSH with 2-bit hashes, our NN query averaged:")
print((l12 + l22 + l32)/3,"seconds of runtime.")

Using LSH with 2-bit hashes, our NN query averaged:
0.13826394081115723 seconds of runtime.


In [231]:
print("Using LSH with 4-bit hashes, our NN query averaged:")
print((l14 + l24 + l34)/3,"seconds of runtime.")

Using LSH with 4-bit hashes, our NN query averaged:
0.11836202939351399 seconds of runtime.


From our KD-Tree, we can see that using a larger leaf-size can help speed up our performance.

From our LSH, we can see that using a larger hash size can help speed up our performance.

Comparing KD-Tree to LSH, our KD-tree performed significantly faster.